In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import scipy.io as sio
import librosa

import time

import skimage
import skimage.io
from skimage.transform import rescale

from scipy.io import wavfile
import os
import subprocess

import sys
sys.path.append("../src")
from spectrogramtools import *
from stego import *
from swpowerstego import *
from trimesh import *


In [ ]:
VPos, _, ITris = load_off("../data/meshseg/391.off")
X = get_hamiltonian_cycle(VPos, ITris)
X = X[:, [2, 1, 0]]
plt.plot(X[:, 0], X[:, 1])
plt.axis("equal")

## Load in audio and compute best target

In [ ]:
# Audio parameters
x, sr = librosa.load("../data/tunes/cowtown.mp3", sr=44100)
win_length = 1024
SX = stft_disjoint(x, win_length)
SX = np.abs(SX)
plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.imshow(SX, aspect='auto')
plt.subplot(212)
plt.plot(np.sum(SX, axis=1))
plt.scatter(np.arange(SX.shape[0]), np.sum(SX, axis=1))

In [ ]:
freq_idxs = [6, 11, 18]
print([int(fi*sr/win_length) for fi in freq_idxs])

# Sliding window parameters
win = 32
fit_lam = 1
q = -1

max_ratio = 0

sp = STFTPowerDisjoint(x, X, win_length, freq_idxs, win, fit_lam, q, viterbi_iters=4)
sp.plot(normalize=False)

sp.solve()
sp.plot(normalize=True)

In [ ]:
y = sp.reconstruct_signal()
ipd.Audio(y, rate=sr)

In [ ]:
prefix = "SpecPower_Win{}_fit{:.3g}_stftwin{}_freqs{}_{}_q{}".format(win, fit_lam, win_length, freq_idxs[0], freq_idxs[1], q)
filename = "{}.mp3".format(prefix)
wavfile.write("out.wav", sr, y)
if os.path.exists(filename):
    os.remove(filename)
subprocess.call("ffmpeg -i out.wav".split() + [filename], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
os.remove("out.wav")

z, sr = librosa.load(filename, sr=sr)
z_sp = STFTPowerDisjoint(z, X, win_length, freq_idxs, win, fit_lam, q)
z_sp.MagSolver.targets = [x for x in sp.MagSolver.targets]
snr = get_snr(x, z)
distortion = z_sp.get_distortion()[0]
ratio = snr/distortion

fhz = [fi*sr/win_length for fi in freq_idxs]
title = "snr = {:.3f}, distortion = {:.3f}, f1 = {:.1f} hz, f2 = {:.1f}hz".format(snr, distortion, fhz[0], fhz[1])

plt.figure(figsize=(6, 6))
z_sp.plot(True, title)
plt.tight_layout()
plt.savefig("{}.png".format(prefix), facecolor='white')


In [ ]:
import polyscope as ps
ps.init()

Z = z_sp.get_signal()
N = Z.shape[0]
edges = np.zeros((N-1, 2))
edges[:, 0] = np.arange(N-1)
edges[:, 1] = np.arange(1, N)
ps.init()
ps.register_curve_network("hamcycle", Z, edges)
ps.show()